In [ ]:
import numpy as np

# import svd from numpy.linalg
from scipy.linalg import svd

import matplotlib.pyplot as plt
import scipy.misc

from scipy.optimize import linear_sum_assignment

In [ ]:
n_points = 10
d = 2

X = np.random.rand(n_points, d)

R = np.random.rand(d, d)
U, _, V = svd(R)
R = U @ V.T
Y = X @ R
Y += 1

In [ ]:
P_gt = np.eye(n_points)[np.random.permutation(n_points)]
Y = P_gt @ Y

S = np.zeros((n_points**2, n_points**2))
for xi in range(n_points):
    p_xi = X[xi, :]
    for yi in range(n_points):
        p_yi = Y[yi, :]
        for xj in range(n_points):
            p_xj = X[xj, :]
            dx = np.linalg.norm(p_xi - p_xj, ord=2)
            for yj in range(n_points):
                p_yj = Y[yj, :]
                dy = np.linalg.norm(p_yi - p_yj, ord=2)
                S[xi * n_points + yi, xj * n_points + yj] = np.exp(-np.abs(dx - dy) / 1e-2)

evals, evecs = np.linalg.eigh(S)
v = evecs[:, np.argmax(evals)]

P = v.reshape(n_points, n_points).T

row_ind, col_ind = linear_sum_assignment(P.max() - P)
P = scipy.sparse.coo_matrix((np.ones(n_points), (row_ind, col_ind)), shape=(n_points, n_points)).toarray()

fig, ax = plt.subplots(2, 2, figsize=(4, 4))
ax[0][0].scatter(X[:, 0], X[:, 1], c="tab:blue")
ax[0][0].scatter(Y[:, 0], Y[:, 1], c="tab:red")
ax[0][0].set_title("point sets")
cax = ax[0][1].imshow(S)
ax[0][1].set_title("similarity matrix")
fig.colorbar(cax, ax=ax[0][1])
ax[1][0].imshow(P_gt)
ax[1][0].set_title("GT Perm.")
ax[1][1].imshow(P)
ax[1][1].set_title("Est. Perm.")